In [1]:
import os
import splitfolders
from tensorflow.keras.preprocessing.image import ImageDataGenerator
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense , Conv2D, MaxPool2D, Flatten, Dropout

In [2]:
# 폴더안의 데이터를 train, test, val로 나누기 -> dataset 폴더안에 나뉘어짐
splitfolders.ratio("./4pose/",output="./dataset/",seed=1337,ratio=(.8,.1,.1))

In [3]:
# 각각 폴더 경로 선언
train_dir = './dataset/train/'
test_dir = './dataset/test/'
val_dir = './dataset/val/'

In [4]:
# 폴더 전처리 한번에 -> ImageDataGenerator
generator = ImageDataGenerator(rescale = 1./255)
train_generator = generator.flow_from_directory(
    directory = train_dir, 
    target_size = (150,150), 
    batch_size = 100, 
    class_mode = 'categorical'
)
test_generator = generator.flow_from_directory(
    directory = test_dir, 
    target_size = (150,150), 
    batch_size = 100, 
    class_mode = 'categorical'
)
val_generator = generator.flow_from_directory(
    directory = val_dir, 
    target_size = (150,150), 
    batch_size = 100, 
    class_mode = 'categorical'
)
# 다중분류 -> categorical 
# 다리 90도 , 고개 90도, 다리 내린자세, 다리 올린자세
print(train_generator.class_indices)

Found 320 images belonging to 4 classes.
Found 40 images belonging to 4 classes.
Found 40 images belonging to 4 classes.
{'90leg': 0, 'head up': 1, 'lay down': 2, 'lay up': 3}


In [5]:
# CNN 모델 선언
model = Sequential()
# 입력층 , Conv층 
model.add(Conv2D(
    filters = 32 , 
    kernel_size = (3,3),
    input_shape = (150,150,3),
    padding='same',
    activation = 'relu'
))

# Pool층
model.add(MaxPool2D(
    pool_size= (2,2)
))
# 특징 과대적합 방지 -> dropout 
model.add(Dropout(0.3))

model.add(Conv2D(
    filters = 16 ,
    kernel_size = (3,3),
    padding='same',
    activation = 'relu'
))
model.add(MaxPool2D(
    pool_size=(2,2) 
))
model.add(Dropout(0.3))
# 특징 추출 끝
# 1차원으로 펼치기 
model.add(Flatten())
# 시작 
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(units=16, activation='relu'))
# 출력 - units -> 특성수 = 4 , 다중분류 -> softmax
model.add(Dense(units=4,activation='softmax'))

In [6]:
# 모델 학습과정 설정하기
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [7]:
#학습
model.fit_generator(
    generator = train_generator,
    epochs = 20,
    validation_data = val_generator
)

C:\Users\smhrd\AppData\Local\Temp\ipykernel_10516\647605993.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/20
4/4 [==============================] - 8s 798ms/step - loss: 1.8705 - accuracy: 0.2937 - val_loss: 1.3702 - val_accuracy: 0.4750
Epoch 2/20
4/4 [==============================] - 3s 639ms/step - loss: 1.4271 - accuracy: 0.3313 - val_loss: 1.3808 - val_accuracy: 0.3250
Epoch 3/20
4/4 [==============================] - 3s 853ms/step - loss: 1.3632 - accuracy: 0.3125 - val_loss: 1.3859 - val_accuracy: 0.2500
Epoch 4/20
4/4 [==============================] - 3s 838ms/step - loss: 1.3690 - accuracy: 0.3219 - val_loss: 1.3862 - val_accuracy: 0.2500
Epoch 5/20
4/4 [==============================] - 3s 589ms/step - loss: 1.3622 - accuracy: 0.3719 - val_loss: 1.3860 - val_accuracy: 0.2500
Epoch 6/20
4/4 [==============================] - 3s 862ms/step - loss: 1.3430 - accuracy: 0.3344 - val_loss: 1.3766 - val_accuracy: 0.2500
Epoch 7/20
4/4 [==============================] - 3s 590ms/step - loss: 1.3168 - accuracy: 0.3375 - val_loss: 1.2572 - val_accuracy: 0.5500
Epoch 8/20
4/4 [====

In [ ]:
from keras.models import load_model
model.save('')